# Chapter 1

- Unsupervised learning : Machine learning on unlabeled data
- pure pattern discovery with unguided learning
- dimension = number of features or columns in dataset
- Algorithms :
    - clustering: normally what a scatterplot shows
        - k-means : 
            - Number of groups formed by samples (rows in dataset)
            - calculates the mean of each cluster as centroid
            - assign nearest centroid to each sample
            - Inertia : how far the samples are from centroids. good clustering has low inertia
    - dimension reduction

### K-Means

```
# Scaling, normalization or standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(samples)
StandardScaler(copy=True, with_mean=True, with_std=True)
samples_scaled = scaler.transform(samples)

from sklearn.cluster import KMeans
model = KMeans(init="random", n_clusters=3, n_init=5, max_iter=100)
model.fit(samples)
labels = model.predict(samples)
#### labels = model.fit_predict(samples) ### Alternative
# See Distribution of samples in a cluster (how well the model performed)
ct = pd.crosstab(df['predicted_labels'], df['target'])
print(ct)
# Cluster quality, Inertia : the spread of clusters
print(model.inertia_)
# Locations of the centroids (You can plot with scatterplot)
centroids = model.cluster_centers_
# Iteration of convergence
print(model.n_iter_)
# Visualize for best cluster : Choose elbow as the optimized k
plt.plot(k_values, inertia_values, marker='o')
plt.show()

### k-means withPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
scaler = StandardScaler()
kmeans = KMeans(n_clusters=3)
pipeline = make_pipeline(scaler, kmeans)
pipeline.fit(samples)
labels = pipeline.predict(samples)
```